In [1]:
import numpy as np
import ospgrillage as og
from IPython import display  # to display images in this notebook
%matplotlib inline

In [ ]:
# Adopted units: N and m
kilo = 1e3
milli = 1e-3
N = 1
m = 1
mm = milli * m
m2 = m ** 2
m3 = m ** 3
m4 = m ** 4
kN = kilo * N
MPa = N / ((mm) ** 2)
GPa = kilo * MPa

# parameters of bridge grillage
L = 33.5 * m  # span
w = 11.565 * m  # width
n_l = 7  # number of longitudinal members
n_t = 11  # number of transverse members
edge_dist = 1.05 * m  # distance between edge beam and first exterior beam
angle = 0  # skew angle

In [ ]:
# define material
concrete = og.create_material(material="concrete", code="AS5100-2017", grade="65MPa")

In [ ]:
edge_longitudinal_section = og.create_section(
    A=0.934 * m2,
    J=0.1857 * m3,
    Iz=0.3478 * m4,
    Iy=0.213602 * m4,
    Az=0.444795 * m2,
    Ay=0.258704 * m2,
)

longitudinal_section = og.create_section(
    A=1.025 * m2,
    J=0.1878 * m3,
    Iz=0.3694 * m4,
    Iy=0.113887e-3 * m4,
    Az=0.0371929 * m2,
    Ay=0.0371902 * m2,
)

transverse_section = og.create_section(
    A=0.504 * m2,
    J=5.22303e-3 * m3,
    Iy=0.32928 * m4,
    Iz=1.3608e-3 * m4,
    Ay=0.42 * m2,
    Az=0.42 * m2,
)

end_transverse_section = og.create_section(
    A=0.504 / 2 * m2,
    J=2.5012e-3 * m3,
    Iy=0.04116 * m4,
    Iz=0.6804e-3 * m4,
    Ay=0.21 * m2,
    Az=0.21 * m2,
)

In [ ]:
# define grillage members
longitudinal_beam = og.create_member(section=longitudinal_section, material=concrete)
edge_longitudinal_beam = og.create_member(
    section=edge_longitudinal_section, material=concrete
)
transverse_slab = og.create_member(section=transverse_section, material=concrete)
end_transverse_slab = og.create_member(
    section=end_transverse_section, material=concrete
)

In [ ]:
# create grillage
simple_grid = og.create_grillage(
    bridge_name="Super-T 33_5m",
    long_dim=L,
    width=w,
    skew=angle,
    num_long_grid=n_l,
    num_trans_grid=n_t,
    edge_beam_dist=edge_dist,
)

In [ ]:
# assign grillage member to element groups of grillage model
simple_grid.set_member(longitudinal_beam, member="interior_main_beam")
simple_grid.set_member(longitudinal_beam, member="exterior_main_beam_1")
simple_grid.set_member(longitudinal_beam, member="exterior_main_beam_2")
simple_grid.set_member(edge_longitudinal_beam, member="edge_beam")
simple_grid.set_member(transverse_slab, member="transverse_slab")
simple_grid.set_member(end_transverse_slab, member="start_edge")
simple_grid.set_member(end_transverse_slab, member="end_edge")

# create the model in OpenSees
simple_grid.create_osp_model(
    pyfile=False
)  # pyfile will not (False) be generated for further analysis (should be create_osp?)
og.opsplt.plot_model("nodes")  # plotting using Get_rendering
og.opsv.plot_model(az_el=(-90, 0),fig_wi_he=(32., 32.))  # plotting using ops_vis
og.plt.show()

In [ ]:
# reference unit load for various load types
P = 1 * kN
# name strings of load cases to be created
static_cases_names = [
    "Line Test Case",
    "Points Test Case (Global)",
    "Points Test Case (Local in Point)",
    "Patch Test Case",
]

# Line load running along midspan width (P is kN/m)
# Create vertical load points in global coordinate system
line_point_1 = og.create_load_vertex(x=L / 2, z=0, p=P)
line_point_2 = og.create_load_vertex(x=L / 2, z=w, p=P)
test_line_load = og.create_load(
    loadtype="line",name="Test Load", point1=line_point_1, point2=line_point_2,
)

# Create load case, add loads, and assign
line_case = og.create_load_case(name=static_cases_names[0])
line_case.add_load(test_line_load)

simple_grid.add_load_case(line_case)

In [ ]:
# Compound point loads along midspan width (P is kN)
# working in global coordinate system
p_list = [
    0,
    edge_dist,
    edge_dist + 2 * m,
    edge_dist + 4 * m,
    edge_dist + 6 * m,
    w - edge_dist,
    w,
]  # creating list of load position

test_points_load = og.create_compound_load(name="Points Test Case (Global)")

# create point load in global coordinate
for p in p_list:
    point = og.create_load(
       loadtype = "point", name ="Point", point1=og.create_load_vertex(x=L / 2, z=p, p=P)
    )
    # add to compound load
    test_points_load.add_load(load_obj=point)

# Create load case, add loads, and assign
points_case = og.create_load_case(name=static_cases_names[1])
points_case.add_load(test_points_load)

simple_grid.add_load_case(points_case)

In [ ]:
simple_grid.analyze()
results = simple_grid.get_results()
# get list of longitudinal element tags along/near mid_span i.e. 84 to 90 in Figure 1
ele_set = list(range(84, 90 + 1))
# query
extracted_bending = results.forces.sel(Loadcase=static_cases_names[1], Element=ele_set, Component="Mz_i")

In [ ]:
# Compound point loads along midspan width
# working in user-defined local coordinate (in point load)
test_points_load = og.create_compound_load(name="Points Test Case (Local in Point)")

# create point load in local coordinate space
for p in p_list:
    point = og.create_load(
        loadtype = "point",name="Point", point1=og.create_load_vertex(x=0, z=p, p=P)
    )
    # add to compound load
    test_points_load.add_load(load_obj=point)

# shift from local to global
test_points_load.set_global_coord(og.Point(L / 2, 0, 0))

# Create load case, add loads, and assign
points_case = og.create_load_case(name=static_cases_names[2])
points_case.add_load(test_points_load)

simple_grid.add_load_case(points_case)

In [ ]:
# Compound point loads along midspan width (P is kN)
# working in global coordinate system
p_list = [
    0,
    edge_dist,
    edge_dist + 2 * m,
    edge_dist + 4 * m,
    edge_dist + 6 * m,
    w - edge_dist,
    w,
]  # creating list of load position

test_points_load = og.create_compound_load(name="Points Test Case (Global)")

# create point load in global coordinate
for p in p_list:
    point = og.create_load(
        loadtype="point", name="Point", point1=og.create_load_vertex(x=L / 2, z=p, p=P)
    )
    # add to compound load
    test_points_load.add_load(load_obj=point)

# Create load case, add loads, and assign
points_case = og.create_load_case(name=static_cases_names[1])
points_case.add_load(test_points_load)

simple_grid.add_load_case(points_case)

In [ ]:
# Compound point loads along midspan width
# working in user-defined local coordinate (in point load)
test_points_load = og.create_compound_load(name="Points Test Case (Local in Point)")

# create point load in local coordinate space
for p in p_list:
    point = og.create_load(
        loadtype = "point",name="Point", point1=og.create_load_vertex(x=0, z=p, p=P)
    )
    # add to compound load
    test_points_load.add_load(load_obj=point)

# shift from local to global
test_points_load.set_global_coord(og.Point(L / 2, 0, 0))

# Create load case, add loads, and assign
points_case = og.create_load_case(name=static_cases_names[2])
points_case.add_load(test_points_load)

simple_grid.add_load_case(points_case)


In [ ]:
# Patch load over entire bridge deck (P is kN/m2)
patch_point_1 = og.create_load_vertex(x=0, z=0, p=P)
patch_point_2 = og.create_load_vertex(x=L, z=0, p=P)
patch_point_3 = og.create_load_vertex(x=L, z=w, p=P)
patch_point_4 = og.create_load_vertex(x=0, z=w, p=P)
test_patch_load = og.create_load(
    loadtype="patch",
    name="Test Load",
    point1=patch_point_1,
    point2=patch_point_2,
    point3=patch_point_3,
    point4=patch_point_4,
)

# Create load case, add loads, and assign
patch_case = og.create_load_case(name=static_cases_names[3])
patch_case.add_load(test_patch_load)
simple_grid.add_load_case(patch_case)

In [ ]:
simple_grid.analyze()

In [ ]:
results = simple_grid.get_results() # gets basic results

In [ ]:
l_factor = 2.3
p_factor = 0.5
# combination with line load case and patch load case
load_combinations = {static_cases_names[0]:l_factor,static_cases_names[-1]:p_factor}
combination_results = simple_grid.get_results(combinations=load_combinations)

In [ ]:
static_cases_names

In [ ]:
# get list of longitudinal element tags along/near mid_span i.e. 84 to 90 in Figure 1
ele_set = list(range(25, 30 + 1))
# query
extracted_bending = results.forces.sel(Loadcase=static_cases_names, Element=ele_set, Component="Mz_i")

In [ ]:
np.sum(
np.array(
    results.forces.sel(
        Loadcase=static_cases_names, Element=ele_set, Component="Mz_i"
    )
),
axis=1,
)

In [ ]:
sum_node_force = np.sum(
    np.array(combination_results.forces.sel(Element=ele_set, Component="Mz_i"))
)

In [ ]:
print(sum_node_force)

In [ ]:
og.plot_force(simple_grid,results, member="exterior_main_beam_2", component="Mz", loadcase = ["Line Test Case"])

In [3]:

# create section of shell element
slab_shell_section = og.create_section(h=0.2) # h = thickness
# shell elements for slab
slab_shell = og.create_member(section=slab_shell_section, material=concrete)
# create grillage with shell model type
example_bridge = og.create_grillage(bridge_name="Shell grillage", long_dim=10, width=7, skew=0,
                                    num_long_grid=6, num_trans_grid=11, edge_beam_dist=1, mesh_type="Orth",
                                    model_type="shell_beam", max_mesh_size_z=0.5, offset_beam_y_dist=0.499,
                                    link_nodes_width=0.89, beam_width = 0.5)
# set shell members to shell elements
example_bridge.set_shell_members(slab_shell)

NameError: name 'Material' is not defined

In [ ]:
og.opsplt.plot_model("nodes")  # plotting using Get_rendering
og.opsv.plot_model(az_el=(-90, 0),fig_wi_he=(32., 32.))  # plotting using ops_vis
og.plt.show()
